<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-finanzas/blob/main/Limpieza_y_transformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Librerias generales
import pandas as pd
import numpy as np
from google.colab import drive
from collections import Counter
import re

In [22]:
 pip install chardet

In [23]:
drive.mount('/content/drive')

df_siniestros= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/DB_Siniestros.txt')
df_sociodemograficas= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/BD_Sociodemograficas.txt', encoding='latin-1',  delimiter='\t')
df_expuestos= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/BD_Expuestos.txt',delimiter='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Data siniestros**

In [24]:
df_siniestros



,Mes_Pago,Asegurado_Id,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,201803,4834968,EXAMENES DE DIAGNOSTICO,Z011,EXAMEN DE OÍDOS Y DE LA AUDICIÓN,1,4.953413e+05
1,201909,14915125,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,4.229947e+05
2,201807,42959949,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,1.085708e+06
3,201803,30540131,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,1,4.131626e+05
4,201905,21442420,LABORATORIO CLINICO,9,DIAGNÓSTICO PENDIENTE,18,1.093855e+06
...,...,...,...,...,...,...,...
3308475,201912,6197783,EXAMENES DE DIAGNOSTICO,9,DIAGNÓSTICO PENDIENTE,1,4.699041e+05
3308476,201905,1001139,CONSULTA EXTERNA,9,DIAGNÓSTICO PENDIENTE,4,7.151695e+05
3308477,201803,21888100,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,7.778954e+05
3308478,201805,56483856,CONSULTA DE URGENCIAS,9,DIAGNÓSTICO PENDIENTE,1,4.322398e+05


In [25]:
# Inspección inicial
print(df_siniestros.info())
print(df_siniestros.isnull().sum()) ### no se evidencian valores nulos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3308480 entries, 0 to 3308479
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Mes_Pago            int64  
 1   Asegurado_Id        int64  
 2   Reclamacion         object 
 3   Diagnostico_Codigo  object 
 4   Diagnostico_Desc    object 
 5   Eventos             int64  
 6   Valor_Pagado        float64
dtypes: float64(1), int64(3), object(3)
memory usage: 176.7+ MB
None
Mes_Pago              0
Asegurado_Id          0
Reclamacion           0
Diagnostico_Codigo    0
Diagnostico_Desc      0
Eventos               0
Valor_Pagado          0
dtype: int64


In [26]:
# Limpia los nombres de las columnas eliminando espacios y caracteres de tabulación
df_siniestros.columns = df_siniestros.columns.str.strip()
### cambiar letra del data por miniscula
df_siniestros = df_siniestros.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [27]:

## convertir columna Mes_Pago	en formato de data time
df_siniestros['Mes_Pago'] = df_siniestros['Mes_Pago'].astype(str)
# Convertir la columna 'fecha' al formato datetime
df_siniestros['Mes_Pago'] = pd.to_datetime(df_siniestros['Mes_Pago'], format='%Y%m')

In [28]:
df_siniestros


,Mes_Pago,Asegurado_Id,Reclamacion,Diagnostico_Codigo,Diagnostico_Desc,Eventos,Valor_Pagado
0,2018-03-01,4834968,examenes de diagnostico,z011,examen de oídos y de la audición,1,4.953413e+05
1,2019-09-01,14915125,examenes de diagnostico,9,diagnóstico pendiente,1,4.229947e+05
2,2018-07-01,42959949,examenes de diagnostico,9,diagnóstico pendiente,1,1.085708e+06
3,2018-03-01,30540131,consulta externa,9,diagnóstico pendiente,1,4.131626e+05
4,2019-05-01,21442420,laboratorio clinico,9,diagnóstico pendiente,18,1.093855e+06
...,...,...,...,...,...,...,...
3308475,2019-12-01,6197783,examenes de diagnostico,9,diagnóstico pendiente,1,4.699041e+05
3308476,2019-05-01,1001139,consulta externa,9,diagnóstico pendiente,4,7.151695e+05
3308477,2018-03-01,21888100,consulta de urgencias,9,diagnóstico pendiente,1,7.778954e+05
3308478,2018-05-01,56483856,consulta de urgencias,9,diagnóstico pendiente,1,4.322398e+05


In [29]:
df_siniestros['Diagnostico_Desc'].value_counts(

)

Diagnostico_Desc
diagnóstico pendiente                                                             2740851
examen de laboratorio                                                              104257
otros controles generales de salud de rutina de otras subpoblaciones definidas      21391
hipotiroidismo, no especificado                                                     19032
hipertensión esencial (primaria)                                                    10563
                                                                                   ...   
pediculosis debida a pediculus humanus capitis                                          1
dermatitis de contacto, forma no especificada, debida a colorantes                      1
alucinaciones auditivas                                                                 1
absceso amebiano del cerebro (g07*)                                                     1
atención materna por otras anormalidades de los órganos pelvianos                  

In [30]:
### conocimiento de las multiples reclamaciones
df_siniestros['Reclamacion'].value_counts(

)

Reclamacion
consulta externa                                                   1255793
examenes de diagnostico                                             694238
laboratorio clinico                                                 520214
consulta de urgencias                                               365872
cirugia ambulatoria por enfermedad                                  106397
fisioterapia ilimitada                                               98370
tratamientos especificos                                             49708
tratamiento medico ambulatorio por accidentes                        39360
consultas prenatales                                                 33685
tratamiento medico hospitalario por enfermedad                       28504
tratamiento quirurgico hospitalario por enfermedad                   20484
cirugia ambulatoria por accidente                                    17594
tratamiento ambulatorio por cancer                                   14949
infiltracione

# **Data Sociodemograficas**

In [31]:
df_sociodemograficas

,Asegurado_Id,Mujer,FechaNacimiento,Ciudad,CANCER,EPOC,DIABETES,HIPERTENSION,ENF_CARDIOVASCULAR
0,18686469,1,17/01/1984,Bogota,0,0,0,0,0
1,16087775,0,12/03/1984,Medellin,0,0,0,0,0
2,35757761,1,19/12/1993,Bogota,0,0,0,0,0
3,48691460,0,14/11/2000,Bogota,0,0,0,0,0
4,36074099,1,6/08/2015,Cali,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
267307,21676160,1,21/01/1988,Bogota,0,0,0,0,0
267308,4782725,1,23/01/1975,Cali,0,0,0,0,0
267309,10100593,1,4/07/1975,Medellin,0,1,0,0,0
267310,34178573,1,15/10/1978,Cali,0,0,0,0,0


In [32]:
# Inspección inicial
print(df_sociodemograficas.info())
print(df_sociodemograficas.isnull().sum()) ### no se evidencian valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267312 entries, 0 to 267311
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Asegurado_Id        267312 non-null  int64 
 1   Mujer               267312 non-null  int64 
 2   FechaNacimiento     267312 non-null  object
 3   Ciudad              267311 non-null  object
 4   CANCER              267312 non-null  int64 
 5   EPOC                267312 non-null  int64 
 6   DIABETES            267312 non-null  int64 
 7   HIPERTENSION        267312 non-null  int64 
 8   ENF_CARDIOVASCULAR  267312 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 18.4+ MB
None
Asegurado_Id          0
Mujer                 0
FechaNacimiento       0
Ciudad                1
CANCER                0
EPOC                  0
DIABETES              0
HIPERTENSION          0
ENF_CARDIOVASCULAR    0
dtype: int64


In [34]:
### cambiar letra del data por miniscula
df_sociodemograficas= df_sociodemograficas.applymap(lambda x: x.lower() if isinstance(x, str) else x)
df_sociodemograficas

# **Data Expuestos**

In [38]:
df_expuestos

,Asegurado_Id,Poliza_Asegurado_Id,FECHA_INICIO,FECHA_CANCELACION,FECHA_FIN
0,16286173,125696117,2018/01/01,2018/04/10,2018/04/10
1,11885152,141320760,2018/01/01,NaN,2019/12/31
2,763957,162532851,2018/01/01,NaN,2019/12/31
3,29082204,154620473,2018/01/01,2018/04/19,2018/04/19
4,4567640,93633862,2018/01/01,NaN,2019/12/31
...,...,...,...,...,...
300895,28476983,53646742,2018/01/01,NaN,2019/12/31
300896,8479396,89963102,2018/01/01,2018/03/08,2018/03/08
300897,6598270,165960557,2018/02/28,NaN,2019/12/31
300898,1000907,105811540,2018/01/01,NaN,2019/12/31


In [39]:
# Inspección inicial
print(df_expuestos.info())
print(df_expuestos.isnull().sum()) ### no se evidencian valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300900 entries, 0 to 300899
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Asegurado_Id         300900 non-null  int64 
 1   Poliza_Asegurado_Id  300900 non-null  int64 
 2   FECHA_INICIO         300900 non-null  object
 3   FECHA_CANCELACION    151963 non-null  object
 4   FECHA_FIN            300900 non-null  object
dtypes: int64(2), object(3)
memory usage: 11.5+ MB
None
Asegurado_Id                0
Poliza_Asegurado_Id         0
FECHA_INICIO                0
FECHA_CANCELACION      148937
FECHA_FIN                   0
dtype: int64
